In [1]:
#親クラスの読み込み
from DP import DataProcessor
from Result import Results
from HR import HorseResults
from Ped import Peds
from ST import ShutubaTable
from JR import JockeyResults
from TR import TrainerResults

In [2]:
#ライブラリをインポート
import pandas as pd
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import optuna.integration.lightgbm as lgb_o
import warnings

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')

In [3]:
r = Results.read_pickle(['data/race_results_11_20.pickle'])
hr = HorseResults.read_pickle(['data/horse_results_all.pickle'])
p = Peds.read_pickle(['data/peds_all.pickle'])
jr = JockeyResults.read_pickle(['data/jockey_results.pickle'])
tr = TrainerResults.read_pickle(['data/trainer_results.pickle'])

In [5]:
r.preprocessing()

In [6]:
r.merge_horse_results(hr)

  0%|          | 0/1080 [00:00<?, ?it/s]

In [7]:
p.encode()
r.merge_peds(p.peds_e)

In [8]:
r.merge_jockey(jr)

  0%|          | 0/1080 [00:00<?, ?it/s]

In [9]:
r.merge_trainer(tr)

  0%|          | 0/1080 [00:00<?, ?it/s]

In [10]:
r.process_categorical() 

In [10]:
r.data_c.head()

,rank,odds,favorite,frame_number,horse_number,age,weight_j,prize,course_len,n_horses,date,horse_id,jockey_id,trainer_id,rank_1R_before,money_1R_before,favorite_1R_before,last_1R_before,odds_1R_before,final_corner_1R_before,rank_2R_before,money_2R_before,favorite_2R_before,last_2R_before,odds_2R_before,final_corner_2R_before,rank_3R_before,money_3R_before,favorite_3R_before,last_3R_before,odds_3R_before,final_corner_3R_before,course_len_all,time_rank_p,interval,course_len_dif,peds_0,peds_4,win_rate_1_1Y_before_j,win_rate_3_1Y_before_j,prize_1Y_before_j,win_rate_1_1Y_before_t,win_rate_3_1Y_before_t,prize_1Y_before_t,race_type_芝,race_type_ダート,race_type_障害,sex_牡,sex_牝,sex_セ,race_park_札幌,race_park_新潟,race_park_小倉,race_park_中山,race_park_阪神,race_park_函館,race_park_京都,race_park_東京,race_park_中京,race_park_福島
201101010101,0,4.7,3.0,2,2,2,54.0,500.0,18,7,2011-08-13,2009105006,01019,00379,8.0,0.0,8.0,34.7,28.8,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,1.0,34.0,4.0,778,1454,0.084,0.240,96905.1,0.062,0.140,16330.6,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0
201101010101,0,3.4,2.0,3,3,2,54.0,500.0,18,7,2011-08-13,2009100577,05203,01061,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,762,1634,0.119,0.333,188832.6,0.138,0.400,52106.2,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0
201101010101,0,2.1,1.0,5,5,2,54.0,500.0,18,7,2011-08-13,2009102908,01093,01089,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,788,1454,0.086,0.262,137964.6,0.051,0.168,23638.6,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0
201101010101,1,7.0,4.0,4,4,2,54.0,500.0,18,7,2011-08-13,2009109159,01095,01075,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,778,1639,0.069,0.194,95243.0,0.087,0.292,108027.3,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0
201101010101,1,17.5,5.0,1,1,2,54.0,500.0,18,7,2011-08-13,2009105263,01127,01026,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,968,1888,0.110,0.284,114614.3,0.062,0.167,23687.4,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0


In [11]:
r.data_e = r.data_c[((r.data_c["rank"]==0) & (r.data_c["odds"]<=32.0)) | (r.data_c["rank"]==1)]

In [12]:
r.data_d = r.data_e[((r.data_e["rank"]==0) & (r.data_e["favorite"]<=10)) | (r.data_e["rank"]==1)]

In [13]:
def split_data(df, test_size=0.3):
    sorted_id_list = df.sort_values("date").index.unique()
    train_id_list = sorted_id_list[: round(len(sorted_id_list) * (1 - test_size))]
    test_id_list = sorted_id_list[round(len(sorted_id_list) * (1 - test_size)) :]
    train = df.loc[train_id_list]
    test = df.loc[test_id_list]
    return train, test

train, test = split_data(r.data_d)

train, valid = split_data(train)

X_train = train.drop(['rank', 'date',"horse_id","jockey_id","trainer_id","favorite","odds"], axis=1)
y_train = train['rank']
X_valid = valid.drop(['rank', 'date',"horse_id","jockey_id","trainer_id","favorite","odds"], axis=1)
y_valid = valid['rank']

#データセットを作成
lgb_train = lgb_o.Dataset(X_train.values, y_train.values)
lgb_valid = lgb_o.Dataset(X_valid.values, y_valid.values)

In [14]:
params = {
    'objective': 'binary', #今回は0or1の二値予測なのでbinaryを指定
    'random_state': 100,
    'metric': 'auc',
    'boosting': 'gbdt'
         }

In [15]:
lgb_clf_o = lgb_o.train(params, lgb_train,
                        valid_sets=(lgb_train, lgb_valid),
                        verbose_eval=100,
                        early_stopping_rounds=10)

[I 2021-11-03 11:00:25,215] A new study created in memory with name: no-name-3a921870-bcb4-439a-9ff9-a3f2e4dc79fb
feature_fraction, val_score: -inf:   0%|          | 0/7 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 173288, number of negative: 40822
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.010074 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4991
[LightGBM] [Info] Number of data points in the train set: 214110, number of used features: 68
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.809341 -> initscore=1.445734
[LightGBM] [Info] Start training from score 1.445734
Training until validation scores don't improve for 10 rounds
[100]	valid_0's auc: 0.78831	valid_1's auc: 0.815649


feature_fraction, val_score: 0.815940:  14%|#4        | 1/7 [00:04<00:25,  4.23s/it]

Early stopping, best iteration is:
[124]	valid_0's auc: 0.79271	valid_1's auc: 0.81594
[LightGBM] [Info] Number of positive: 173288, number of negative: 40822
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.010189 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4991
[LightGBM] [Info] Number of data points in the train set: 214110, number of used features: 68
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.809341 -> initscore=1.445734
[LightGBM] [Info] Start training from score 1.445734
Training until validation scores don't improve for 10 rounds
[100]	valid_0's auc: 0.788223	valid_1's auc: 0.815098


feature_fraction, val_score: 0.815940:  29%|##8       | 2/7 [00:08<00:20,  4.05s/it]

...

Early stopping, best iteration is:
[207]	valid_0's auc: 0.797052	valid_1's auc: 0.817682
[LightGBM] [Info] Number of positive: 173288, number of negative: 40822
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006873 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4991
[LightGBM] [Info] Number of data points in the train set: 214110, number of used features: 68
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.809341 -> initscore=1.445734
[LightGBM] [Info] Start training from score 1.445734
Training until validation scores don't improve for 10 rounds
[100]	valid_0's auc: 0.783584	valid_1's auc: 0.815507
[200]	valid_0's auc: 0.796036	valid_1's auc: 0.817099


min_data_in_leaf, val_score: 0.817682:  80%|########  | 4/5 [00:22<00:05,  5.91s/it]

Early stopping, best iteration is:
[232]	valid_0's auc: 0.799337	valid_1's auc: 0.817342
[LightGBM] [Info] Number of positive: 173288, number of negative: 40822
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008511 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4991
[LightGBM] [Info] Number of data points in the train set: 214110, number of used features: 68
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.809341 -> initscore=1.445734
[LightGBM] [Info] Start training from score 1.445734
Training until validation scores don't improve for 10 rounds
[100]	valid_0's auc: 0.783269	valid_1's auc: 0.815676


min_data_in_leaf, val_score: 0.817682: 100%|##########| 5/5 [00:28<00:00,  5.60s/it]

Early stopping, best iteration is:
[178]	valid_0's auc: 0.793674	valid_1's auc: 0.81736


In [16]:
lgb_clf_o.params#確認

{'objective': 'binary',
 'random_state': 100,
 'metric': 'auc',
 'boosting': 'gbdt',
 'feature_pre_filter': False,
 'lambda_l1': 9.327802059899733,
 'lambda_l2': 0.00010111708372666778,
 'num_leaves': 31,
 'feature_fraction': 0.4,
 'bagging_fraction': 1.0,
 'bagging_freq': 0,
 'min_child_samples': 50,
 'num_iterations': 1000,
 'early_stopping_round': 10}

In [17]:
train, test = split_data(r.data_d)

In [18]:
#説明変数と目的変数に分ける。dateはこの後不要なので省く。
X_train = train.drop(["rank", "date","horse_id","jockey_id","trainer_id","favorite","odds"], axis=1)
y_train = train['rank']
X_test = test.drop(["rank", "date","horse_id","jockey_id","trainer_id","favorite","odds"], axis=1)
y_test = test['rank']

In [19]:
lgb_clf_o.params2={
'objective': 'binary',
 'random_state': 100,
 'metric': 'auc',
 'boosting': 'gbdt',
 'feature_pre_filter': False,
 'lambda_l1': 9.327802059899733,
 'lambda_l2': 0.00010111708372666778,
 'num_leaves': 31,
 'feature_fraction': 0.4,
 'bagging_fraction': 1.0,
 'bagging_freq': 0,
 'min_child_samples': 50,
 'num_iterations': 1000,
}

In [20]:
lgb_clf = lgb.LGBMClassifier(**lgb_clf_o.params2)
lgb_clf.fit(X_train.values, y_train.values)

[LightGBM] [Warning] bagging_freq is set=0, subsample_freq=0 will be ignored. Current value: bagging_freq=0
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.4, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] lambda_l2 is set=0.00010111708372666778, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.00010111708372666778
[LightGBM] [Warning] lambda_l1 is set=9.327802059899733, reg_alpha=0.0 will be ignored. Current value: lambda_l1=9.327802059899733


LGBMClassifier(bagging_fraction=1.0, bagging_freq=0, boosting='gbdt',
               feature_fraction=0.4, feature_pre_filter=False,
               lambda_l1=9.327802059899733, lambda_l2=0.00010111708372666778,
               metric='auc', min_child_samples=50, num_iterations=1000,
               objective='binary', random_state=100)

In [21]:
from sklearn.metrics import accuracy_score
print('正解率(test)',accuracy_score(y_test, lgb_clf.predict(X_test,axis=1)))

正解率(test) 0.8185615070683606


In [22]:
roc_auc_score(y_test,lgb_clf.predict(X_test,axis=1))

[LightGBM] [Warning] Unknown parameter: axis


0.6275930673902916